In [5]:
import json
import string
import xml.dom.minidom
from collections import defaultdict

import sys
import xlrd

import numpy as np
import pandas as pd
import sqlite3 as sq

from  openpyxl.workbook  import  Workbook
from  openpyxl.writer.excel import ExcelWriter

from YouDaoSpider import YoudaoSpider

In [2]:
def xml2json(name, word_dict, level):
    import json

    dom = xml.dom.minidom.parse(name)
    root = dom.documentElement
    items = root.getElementsByTagName('item')
    # print len(items)

    c = 0
    for item in items:

        word = item.getElementsByTagName('word')[0].firstChild.data
        if word in word_dict:
            if level not in word_dict[word]['test']:
                word_dict[word]['test'] += ' ' + level
            continue

        trans = item.getElementsByTagName('trans')[0].firstChild.data
        phonetic = item.getElementsByTagName('phonetic')[0].firstChild.data

        mean_list, example_list = parse_tran(trans)
        mean_str = '\n'.join(mean_list)
        example_str = '\n'.join(example_list)

        word_dict[word] = {'phonetic': phonetic, 'mean': mean_str, 'example': example_str, 'test': level}



In [3]:

def is_meaning(line):
    if line[0] in string.letters and '.' in line:
        return True
    return False
        
def parse_tran(tran):
    lines = tran.split('\n')

    mean_list = []
    example_list = []
    method = None
    
    for line in lines:
        if not line:
            continue

        if is_meaning(line):
            mean_list.append(line)
        elif line.startswith(u'【'):
            method = line.split(u'】')[1].strip()
        elif line[0] in string.letters and ':' in line and ' ' in line.split(':')[0]:
            example = line.split(':')
            example_list.append(example[1] + ': '+ example[0])

    # print len(mean_list), len(example_list)
    return mean_list, example_list
# tran = trans[0].firstChild.data
# parse_tran(tran)

In [4]:

word_dict = {}

xml2json('700CET4.xml', word_dict, 'CET4')

print len(word_dict)

xml2json('CET-4 EASY.xml', word_dict, 'CET4')

print len(word_dict)

xml2json('CET-4 HARD.xml', word_dict, 'CET4')

print len(word_dict)

xml2json('CET-6.xml', word_dict, 'CET6')

print len(word_dict)

xml2json('POST CET-6.xml', word_dict, 'CET6')

print len(word_dict)

IOError: [Errno 2] No such file or directory: '700CET4.xml'

In [42]:
def trim_sqlite_example(content):
    lines = content.split('\n')
    examples = []
    for line in lines:
        if not line:
            continue
        t = line.replace('/r', '').replace('/n', '')
        if '    ' not in t:
            continue
        p = t.index('    ')
        examples.append(t[p+1:].strip() + ':' + t[:p].strip())
    return '\n'.join(examples)

In [60]:
xml2json('TOFEL.xml', word_dict, 'TOFEL')

print len(word_dict)

xml2json('tofel.xml', word_dict, 'TOFEL')

print len(word_dict)

7125
7125


In [68]:
def merge_excel(word_dict, fname, level):
    data = xlrd.open_workbook(fname)
    table = data.sheets()[0]
    nrows = table.nrows # 获取表的行数
    for i in range(nrows): # 循环逐行打印
        row = table.row_values(i)
        row[7][0] = '['
        row[7][-1] = ']'
        if row[0] not in word_dict:
            word_dict[row[0]] = {'phonetic': row[7], 'mean': row[4], 'test': 'GRE'}
            word_dict[row[0]]['example'] = row[6] if row[6] else '' 
        else:
            word_dict[row[0]]['mean'] += '\n' + row[4]
            if word_dict[row[0]]['example'] and row[6]:
                word_dict[row[0]]['example'] += "\n" + row[6]
            else:
                word_dict[row[0]]['example'] = row[6]
            if level not in word_dict[row[0]]['test']:
                word_dict[row[0]]['test'] += ' ' + level
                
        word_dict[row[0]]['lenovo'] = row[2] if row[2] else ''
        word_dict[row[0]]['etyma'] = row[3] if row[3] else ''
        

In [61]:
merge_excel(word_dict, 'GRE.xls', 'GRE')
merge_excel(word_dict, '3000.xls', 'GRE')
len(word_dict)  

12867

In [62]:
import HTMLParser
con = sq.connect("cet6.sqlite3")
cursor = con.cursor()
rset= cursor.execute("select * from Note")
html_parser = HTMLParser.HTMLParser()
c = 0
for item in rset.fetchall():
    w = item[0].lower()
    m = html_parser.unescape(item[1])
    s = html_parser.unescape(item[2] if item[2] else '')
    s = trim_sqlite_example(s)
    if w in word_dict and s:
        if word_dict[w]['example']:
            word_dict[w]['example'] += "\n" + s
        else:
            word_dict[w]['example'] = s
        if 'CET6' not in word_dict[w]['test']:
            word_dict[w]['test'] += ' ' + 'CET6'
        continue
    c += 1
    word_dict[w] = {'phonetic': '', 'mean': m, 'example': s, 'test': 'CET6'}
len(word_dict)      

19093

In [63]:
def json2df(word_dict):
    l = ['word', 'level', 'lenovo', 'etyma', 'meanZh', 'meanEn', 'example', 'phonetic', 'test']
    content = []
    for w in word_dict:
        row = [''] * 9
        row[0] = w
        row[1] = 0
        row[2] = word_dict[w]['lenovo'] if 'lenovo' in word_dict[w] else ''
        row[3] = word_dict[w]['etyma'] if 'etyma' in word_dict[w] else ''
        row[4] = word_dict[w]['mean']
        row[6] = word_dict[w]['example']
        row[7] = word_dict[w]['phonetic']
        row[8] = word_dict[w]['test']
        content.append(row)

    print len(content)
    return pd.DataFrame(content, columns=l)

In [64]:
data = json2df(word_dict)

19093


In [65]:
def save_to_excel(data, fname):
    writer = pd.ExcelWriter(fname)
    data.to_excel(writer, index=False)
    writer.save()

In [66]:
save_to_excel(data, 'gloss.xls')